In [1]:
from datalib import BinanceTradeLoader 
import math
import numpy as np
import pandas as pd
import seaborn as sns
from calendar import monthrange
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import asyncio

In [2]:
resample_interval = 1000 #milliseconds
trade_intervals = 60 #resample_intervals
local_cache_path = "/media/corvin/BigData/Crypto/binance/"
trade_loader = BinanceTradeLoader(local_cache_path)

In [3]:
def make_monthly_volume_vwap_and_imbalance(symbol, month, year, resample_interval) :
    num_days = monthrange(year, month)[1]
    return pd.concat(make_daily_volume_vwap_and_imbalance(symbol, day, month, year, resample_interval) for day in range(1,num_days + 1)).reset_index(drop = True)

def make_daily_volume_vwap_and_imbalance(symbol, day, month, year, resample_interval) :
    trades = trade_loader.load_daily_trades_usdt_margined(symbol, day, month, year)
    resampled = make_volume_vwap_and_imbalance(trades, resample_interval)
    del trades
    return resampled
def make_volume_vwap_and_imbalance(trades, resample_interval = 1000) :
    trades["buy_usdt_volume"] = trades["price"] * trades["qty"] *(1 - trades["isBuyerMaker"])
    trades["sell_usdt_volume"] = trades["price"] * trades["qty"] *trades["isBuyerMaker"]
    trades["usdt_volume"] = trades["price"] * trades["qty"]
    resampled =  trades.resample(f'{resample_interval}ms', on = "time").agg({"buy_usdt_volume" : np.sum, "sell_usdt_volume" : np.sum, "usdt_volume" : np.sum, "qty" : np.sum}).reset_index()
    resampled["buy_percentage"] = resampled["buy_usdt_volume"] / (resampled["buy_usdt_volume"] + resampled["sell_usdt_volume"])
    resampled["vwap"] = resampled["usdt_volume"] / resampled["qty"]
    resampled["vwap"].fillna(method="ffill", inplace = True)
    resampled["buy_percentage"].fillna(0.5, inplace = True)
    return resampled.drop(columns = ["usdt_volume", "buy_usdt_volume", "sell_usdt_volume"])


In [5]:
import time
start_time = time.time()
make_monthly_volume_vwap_and_imbalance("EOS", 6,2021, resample_interval)
print ("My program took", time.time() - start_time, "to run")

My program took 25.672229051589966 to run


In [6]:

test = make_monthly_volume_vwap_and_imbalance("EOS", 6,2021, resample_interval)
# test.isna().sum()
# test["vwap"][test["vwap"].isna()]
test

,time,qty,buy_percentage,vwap
0,2021-06-01 00:00:01,8585.1,0.393871,6.625941
1,2021-06-01 00:00:02,539.3,0.092733,6.627147
2,2021-06-01 00:00:03,2668.9,0.261813,6.626881
3,2021-06-01 00:00:04,5523.9,0.653377,6.626320
4,2021-06-01 00:00:05,722.7,0.028093,6.625028
...,...,...,...,...
2591972,2021-06-30 23:59:55,646.6,0.997990,4.135464
2591973,2021-06-30 23:59:56,4829.3,0.821752,4.136462
2591974,2021-06-30 23:59:57,1.3,0.000000,4.137000
2591975,2021-06-30 23:59:58,265.4,1.000000,4.138225


In [7]:
test["1_min_qty"] = test["qty"].rolling(60).sum()
test["1_min_vwap"] = (test["qty"] * test["vwap"]).rolling(60).sum() / test["1_min_qty"]
test["1_min_buy_percentage"] = (test["qty"] * test["buy_percentage"]).rolling(60).sum() / test["1_min_qty"]
# test.dropna(inplace= True)
test["1_hour_qty"] = test["qty"].rolling(60 * 60).sum()
test["1_hour_vwap"] = (test["qty"] * test["vwap"]).rolling(60 * 60).sum() / test["1_min_qty"]
test["1_hour_buy_percentage"] = (test["qty"] * test["buy_percentage"]).rolling(60 * 60).sum() / test["1_min_qty"]

test.dropna()

,time,qty,buy_percentage,vwap,1_min_qty,1_min_vwap,1_min_buy_percentage,1_hour_qty,1_hour_vwap,1_hour_buy_percentage
3599,2021-06-01 01:00:00,360.3,0.996670,6.685066,45227.3,6.684334,0.444562,7417328.1,1094.064905,76.989220
3600,2021-06-01 01:00:01,208.5,0.139572,6.683325,45200.0,6.684341,0.440346,7408951.5,1093.498026,76.961553
3601,2021-06-01 01:00:02,584.7,0.907317,6.683440,44899.7,6.684451,0.455106,7408996.9,1100.819039,77.486992
3602,2021-06-01 01:00:03,5.3,0.000000,6.684000,44120.2,6.684596,0.463065,7406333.3,1119.867850,78.840167
3603,2021-06-01 01:00:04,18.6,0.000000,6.684000,42917.7,6.684799,0.448520,7400828.0,1150.395168,80.965074
...,...,...,...,...,...,...,...,...,...,...
2591972,2021-06-30 23:59:55,646.6,0.997990,4.135464,23816.1,4.133709,0.409248,3524066.6,608.448888,72.037402
2591973,2021-06-30 23:59:56,4829.3,0.821752,4.136462,28539.5,4.134170,0.480568,3527137.2,508.196865,60.214635
2591974,2021-06-30 23:59:57,1.3,0.000000,4.137000,23431.8,4.134006,0.577586,3525812.5,618.743913,73.283822
2591975,2021-06-30 23:59:58,265.4,1.000000,4.138225,23589.7,4.134058,0.584970,3523860.0,614.265269,72.710703
